In [1]:
from minio.error import S3Error
from datetime import datetime
from io import BytesIO
import pandas as pd
import import_ipynb
import sys
import os
from dotenv import load_dotenv
from pathlib import Path
import import_ipynb
from Utils import * #conectar_minio, guardar_df_en_minio, generar_ruta_fecha, extraer_archivo_minio

2025/10/10


In [2]:
# Cargar variables del archivo .env
env_path = Path.cwd().parent.parent / 'notebooks_etl' / '.env'
load_dotenv(dotenv_path=env_path)

True

In [3]:
minio_client = conectar_minio(
    endpoint=os.getenv("MINIO_ENDPOINT"),
    access_key=os.getenv("MINIO_ACCESS_KEY"),
    secret_key=os.getenv("MINIO_SECRET_KEY"),
    secure=False  # Usar HTTPS
)

Conexión exitosa a MinIO en localhost:9000
Buckets disponibles: ['yachay-bronze', 'yachay-golden', 'yachay-landing', 'yachay-silver']


In [4]:
# Listar todos los archivos en difuntos/
objects = minio_client.list_objects("yachay-landing", prefix="difuntos/", recursive=True)

# Filtrar por extensión (ajusta según tus archivos)
archivos_difuntos = [obj.object_name for obj in objects if obj.object_name.lower().endswith('.xlsx')]

print(f"Archivos encontrados: {archivos_difuntos}")

# Procesar cada archivo y consolidar datos
resultados = []
for ruta_archivo in archivos_difuntos:
    try:
        # Extraer datos del archivo actual
        df_temp = extraer_archivo_minio(
            minio_client=minio_client,
            bucket_name="yachay-landing",
            ruta_archivo=ruta_archivo,
            tipo_archivo="excel"  # o el tipo correspondiente
        )
        
        # Agregar columna con nombre del archivo si es necesario
        df_temp['archivo_origen'] = ruta_archivo
        
        resultados.append(df_temp)
        
    except Exception as e:
        print(f"Error procesando {ruta_archivo}: {e}")

# Consolidar todos los DataFrames
if resultados:
    df_datos = pd.concat(resultados, ignore_index=True)
    print(f"Datos consolidados de {len(resultados)} archivos")
else:
    df_datos = pd.DataFrame()


Archivos encontrados: ['difuntos/Defunciones RUAF-ND Pasto del 01-01-2024 al 31-12-2024 (2).xlsx']


Datos consolidados de 1 archivos


In [5]:
ruta_fecha = generar_ruta_fecha()

In [6]:
# Guardar el DataFrame en MinIO
ruta_minio = guardar_df_en_minio(
    minio_client=minio_client,  # Cliente de la conexión anterior
    df=df_datos,
    bucket_name="yachay-bronze",
    ruta_destino=f"Difuntos/{ruta_fecha}/difuntos",  # La extensión se agregará automáticamente
    formato='csv',  # También puede ser 'csv', 'json' o 'excel'
    crear_bucket=True
)

DataFrame guardado exitosamente en: yachay-bronze/Difuntos/2025/10/10/difuntos.csv
